In [1]:
using LinearAlgebra
using Statistics
using Plots
using LaTeXStrings

In [2]:
include("ED_functions_spinless_fermions.jl")

fermi_dirac_distribution (generic function with 1 method)

The purpose of this file is to study if we should change the interaction of the mesoscopic method or not.

In the Giant Rectification article we have:

$$ $$

In [3]:
function Build_H_Giant_Rectification(N, Δ, μ, E, J = 1.0)
    #J: Hopping amplitude.
    #Δ: Nearest-neighbor density-density interaction.
    #μ: Chemical potential of the diode.
    #E: Tilt strength.

    H = zeros(2^N, 2^N)
    
    for i=1:N-1 #H_S
        H += 0.5*J*Build_C_dagi_Cj_Matrix(N, i, i+1)
        H += 0.5*J*Build_C_dagi_Cj_Matrix(N, i+1, i)
        H += Δ*(Build_C_dagi_Cj_Matrix(N, i, i)-0.5*Enlarge_Matrix_site_j(N, 1, Identity))*(Build_C_dagi_Cj_Matrix(N, i+1, i+1)-0.5*Enlarge_Matrix_site_j(N, 1, Identity)) #Interaction 1 (original from paper)
    end

    for i=1:N
        H += (μ + 0.5*E*i)*(Build_C_dagi_Cj_Matrix(N, i, i)-0.5*Enlarge_Matrix_site_j(N, 1, Identity))
    end
        
    return H
end

function Build_H_wMesoscopicInteraction(N, Δ, μ, E, J = 1.0)
    #J: Hopping amplitude.
    #Δ: Nearest-neighbor density-density interaction.
    #μ: Chemical potential of the diode.
    #E: Tilt strength.

    H = zeros(2^N, 2^N)
    
    for i=1:N-1 #H_S
        H += 0.5*J*Build_C_dagi_Cj_Matrix(N, i, i+1)
        H += 0.5*J*Build_C_dagi_Cj_Matrix(N, i+1, i)
        H += Δ*Build_C_dagi_Cj_Matrix(N, i, i)*Build_C_dagi_Cj_Matrix(N, i+1, i+1) #Interaction 2 (the one used in the mesoscopic method)
    end

    for i=1:N
        H += (μ + 0.5*E*i)*(Build_C_dagi_Cj_Matrix(N, i, i)-0.5*Enlarge_Matrix_site_j(N, 1, Identity))
    end
        
    return H
end

zero = [1; 0]
one = [0; 1];

In [4]:
N = 4
Δ = 5

ψ_s1 = kron(one, zero, zero, zero)
ψ_s2 = kron(zero, one, zero, zero)
ψ_s3 = kron(zero, zero, one, zero)
ψ_s4 = kron(zero, zero, zero, one)

E_values = LinRange(0, 12, 200)

Energy_levels_H0 = Dict()
Energy_levels_H1 = Dict()
Energy_levels_H2 = Dict()

for E = E_values
    μ = -E*(N+1)/4
    H0 = Build_H_Giant_Rectification(N, 0.0, μ, E);
    H1 = Build_H_Giant_Rectification(N, Δ, μ, E);
    H2 = Build_H_wMesoscopicInteraction(N, Δ, μ, E);
    
    Energy_levels_H0[E] = real([adjoint(ψ)*H0*ψ for ψ=[ψ_s1, ψ_s2, ψ_s3, ψ_s4]])
    Energy_levels_H1[E] = real([adjoint(ψ)*H1*ψ for ψ=[ψ_s1, ψ_s2, ψ_s3, ψ_s4]])
    Energy_levels_H2[E] = real([adjoint(ψ)*H2*ψ for ψ=[ψ_s1, ψ_s2, ψ_s3, ψ_s4]])   
end

In [205]:
for E = E_values
    μ = -E*(N+1)/4
    E_value = round(E; digits = 3)
    scatter(1:1:N, real(Energy_levels_H0[E]), label = L"H_{int} = 0", m = :hexagon, alpha = 0.6, markersize = 8)
    scatter!(1:1:N, Energy_levels_H1[E], label = L"H_{int} = \Delta (n_{j} -0.5)(n_{j+1} -0.5)", m = :diamond, alpha = 0.6, markersize = 8)
    scatter!(1:1:N, Energy_levels_H2[E], label = L"H_{int} = \Delta n_{j}n_{j+1}", m = :star5, alpha = 0.8, markersize = 6)
    
    plot!(xlabel = "site j", ylabel = L"\langle j | H | j \rangle", title  = "E = $E_value", legend=:outertopleft, size = (1000, 500), margin=5Plots.mm)
    savefig("Results E = $E_value.pdf")
end

Now let's plot the energy levels that I have been using until now for the mesoscopic approach.